# config

In [ ]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [ ]:
import sys
# Add the notebook directory to the Python path
sys.path.append('/kristof.meszaros/pract')
import os
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
import xml.etree.ElementTree as ET
from helpers.meta_functions import remove_all_null_column_pairs_and_unit_cols, read_in_all_data, \
get_category_name_and_description, add_main_df_attributes, print_first_x_dict_elements, get_column_metadata_for_category
from helpers.config import categories_xml_filepath, feature_xml_filepath, data_dict_filepath, feather_path
from helpers import config
from helpers.eda import plot_missing_values, missing_heatmap, draw_dendogram
import missingno as msno

# Parse the XML file
categories_xml_filepath = '../IceCat_Full/IceCat Specifications/CategoriesList.xml'
feature_xml_filepath = '../IceCat_Full/IceCat Specifications/CategoryFeaturesList.xml'
data_dict_filepath = "../IceCat_Full/IceCat Dictionary.xml"
feather_path = '../IceCat_Full/feather/'
category_tree = ET.parse(categories_xml_filepath)
config.category_root = category_tree.getroot()

# read in feather files, get some metadata

## category description and name test

In [ ]:
mice_category_id = 195 # know from below
name, description = get_category_name_and_description(mice_category_id)
print(f"Name: {name},\nDescription: {description}")

##  summary of read-in datasets

In [ ]:
df_dict = read_in_all_data(feather_path, num_files=None)

In [ ]:
def add_main_df_attributes(df_dict):
    """
    Extracts metadata for each dataframe in a dictionary and returns a dataframe containing the metadata
    for all input dataframes, sorted by the number of rows.

    Args:
    - df_dict (dict): A dictionary of dataframes, where each key is a category ID and the corresponding
                      value is a pandas DataFrame.

    Returns:
    - metadata_df (pandas DataFrame): A DataFrame containing metadata for each input DataFrame.
      The metadata includes:
        - id: The key of the DataFrame in the input dictionary
        - category: The value of the 'category_label' column of the first row of the DataFrame
        - rows: The number of rows in the DataFrame
        - columns: The number of columns in the DataFrame
        - metric_col_count: The number of metric columns
        - description: A description of the category, obtained using the `get_category_name_and_description` function.

    Example usage:
    ```
    metadata = add_main_df_attributes({'cat1': df1, 'cat2': df2})
    ```
    """
    have_data = []
    num_of_items=len(df_dict)
    i=0
    for cat_id, df in df_dict.items():
        i+=1
        print(f"{i}/{num_of_items}")
        cat_name, description = get_category_name_and_description(cat_id)
        unit_cols = [col for col in df.columns if "." in col]
        rowcount, colcount = df.shape[0], df.shape[1]
        metric_col_count = len(unit_cols)
        if colcount > 0: assert colcount - 4 == metric_col_count
        rowcounts = df.count()
        num_cols_with_30 = sum(rowcounts >= 30)
        num_cols_with_50 = sum(rowcounts >= 50)
        num_cols_with_100 = sum(rowcounts >= 100)
        have_data.append([cat_id, cat_name, rowcount, metric_col_count, description,
                          num_cols_with_30, num_cols_with_50, num_cols_with_100,
                         num_cols_with_30 + num_cols_with_50 + num_cols_with_100])

    metadata_df = pd.DataFrame(have_data, columns=["id", "category", "rows", "feature_columns", "description",
                                                   "30_non_null", "50_non_null", "100_non_null"])
    metadata_df.sort_values(by=["rows"], ascending=False, inplace=True, ignore_index=True)
    return metadata_df
df_attributes = add_main_df_attributes(df_dict)
df_attributes.head()

In [ ]:
df_attributes[70:81]

In [ ]:
df_attributes[df_attributes.id==2917]

In [ ]:
 df_dict[788].head()

In [ ]:
print(2)
df_attributes["rowcount"] = df_attributes["50_non_null"] * df_attributes["rows"]
to_divide_df = df_attributes[["id","category","rowcount"]].sort_values(
    by="rowcount", ascending=False,ignore_index=True)
df = to_divide_df[to_divide_df.rowcount>0]

In [ ]:
df

In [ ]:
import pandas as pd
df=df.iloc[20:50]
print(df.head())
# assuming df is the dataframe with columns "id" and "rowcount"

# calculate the cumulative rowcount for each id in descending order
df['cumulative_rowcount'] = df['rowcount'].cumsum()

# calculate the total rowcount for all ids
total_rowcount = df['rowcount'].sum()

# calculate the target rowcount for each list
target_rowcount = total_rowcount // 3

# initialize variables for tracking the current list and rowcount
current_list = 0
current_rowcount = 0

# initialize an empty list for each of the 10 lists
lists = [[] for _ in range(3)]

# loop over the ids in descending order of rowcount
for i, row in df.iterrows():
    # add the id to the current list
    lists[current_list].append(row['id'])
    
    # update the current rowcount
    current_rowcount += row['rowcount']
    
    # if the current rowcount is greater than or equal to the target rowcount,
    # move to the next list and reset the current rowcount
    if current_rowcount >= target_rowcount:
        current_list += 1
        current_rowcount = 0
        
    # if we've reached the last list, add the remaining ids to it
    if current_list == 10:
        lists[current_list-1].extend(df.loc[i+1:, 'id'].tolist())
        break


In [ ]:
counter=7
for i in lists:
    print(f"CAT_IDS_{counter}=(",' '.join(str(x) for x in i),")", sep="")
    counter+=1

In [ ]:
lists[0]

In [ ]:
df_attributes[df_attributes.rows>=36]["id"].to_dict()

# EDA
## missing% for each feature

In [ ]:
plot_missing_values(df_dict[195])

## heatmap 1

!/usr/share/miniconda2/envs/py39/bin/pip install missingno --user

In [ ]:
missing_heatmap(df_dict[195], 0, 200, 60, 80)

## heatmap 2

In [ ]:
df_dict[195].iloc[:,60:80]

In [ ]:
df_dict[195]["Battery type.908"].unique()

In [ ]:
msno.heatmap(df_dict[195].iloc[:,60:80])

## dendogram

In [ ]:
draw_dendogram(df_dict[195], 60, 80)

# impute with MICE

create an sklearn pipeline in Python. df is a dataframe with numerical and categorical columns, with lots of existing missing values

1. drop all columns which have less than 50 non-null values, replace df with this.
2. in every single column randomly replace 20% of the non-null values with null, and make this a new dataframe. These newly generated null values will be the target values, we are trying to impute these, and see how good the imputation is! The original null values are irrelevant, I only want to evaluate the pipeline based on the imputation of these newly created null values.
3. standard scale the numerical columns, and one-hot encode the categorical ones
4. Use MICE imputation to impute the numerical columns
5. Use KNNImputer to impute the categorical columns
6. evaluate

In [ ]:
df=df_dict[mice_category_id]
df.head()

#  example SKLEARN pipeline

## create dummy data

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from fancyimpute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score

# create a sample dataframe with missing values
random_state = 123
np.random.seed(random_state)
n_rows = 100
n_missing1 = 55
n_missing2 = 60
col1 = np.random.rand(n_rows)
col2 = col1*2
col3 = np.random.choice(['A', 'B', 'C', 'D'], size=n_rows)
col4 = np.random.choice(['X', 'Y', 'Z'], size=n_rows)
col5 = np.random.choice(['M', 'F', 'Unknown'], size=n_rows)
col1[:n_missing1] = np.nan
col5[:n_missing2] = np.nan
data = {'Numeric1': col1,
        'Numeric2': col2,
        'Category1': col3,
        'Category2': col4,
        'Category3': col5}
data = {k: [np.nan if x == 'nan' else x for x in v] for k, v in data.items()}
df = pd.DataFrame(data)
threshold = 90

# 1. Drop columns with more than 'threshold' missing values
df = df.dropna(thresh=df.shape[0] - threshold, axis=1)
df

## replace 20% of non-nulls with null

In [ ]:
# 2. For every single column randomly replace 20% of the non-null values with null
mask_df = df.copy()
pct_replace = 0.2
replace_indices_dict={}
for col in mask_df.columns:
    non_null_values = mask_df[col].dropna()
    n_replace = int(pct_replace * len(non_null_values))
    replace_indices = np.random.choice(non_null_values.index, size=n_replace, replace=False)
    replace_indices_dict[col] = replace_indices
    mask_df.loc[replace_indices, col] = np.nan
mask_df

In [ ]:
replace_indices_dict

## impute numerical cols
if all cols are NULL for a row, it will go for MeanInmputation as I see

In [ ]:
#train_df, test_df = train_test_split(mask_df, test_size=0.2, random_state=random_state)

num_cols = df.columns[df.columns.str.contains('Numeric')].to_list()
cat_cols = df.columns[df.columns.str.contains('Category')].to_list()

# 3.standard scale the numerical columns, use MICE imputation and evaluate
num_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('splitter', 'passthrough'),
    ('imputer', IterativeImputer())
])
num_pipeline.fit(mask_df[num_cols])
num_transformed = num_pipeline.transform(mask_df[num_cols])
num_transformed

In [ ]:
num_imputed = num_pipeline.named_steps['scaler'].inverse_transform(num_transformed)
num_imputed = pd.DataFrame(num_imputed, columns=num_cols)
num_imputed

In [ ]:
df.loc[replace_indices_dict['Numeric1'],'Numeric1']

In [ ]:
for col in num_cols:
    rmse = mean_squared_error(df.loc[replace_indices_dict[col], col],
                          num_imputed.loc[replace_indices_dict[col],col], squared=False)
    mae = mean_absolute_error(df.loc[replace_indices_dict[col],col],
                          num_imputed.loc[replace_indices_dict[col],col])
    print(f"{col}, RMSE: {rmse}, MAE: {mae}, number of test cases: {len(replace_indices_dict[col])}")

In [ ]:
df[num_cols] = num_imputed
df[num_cols] = StandardScaler().fit_transform(df[num_cols])
df.head()

# categorical


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Define the percentage of categorical values to replace with null
null_percentage = 0.2

# Loop through each categorical column
# LOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOP
#for cat_col in cat_cols:
cat_col='Category1'
# Replace a percentage of the categorical column with null
null_indices = df[cat_col].dropna().sample(frac=null_percentage).index
y_test = df.loc[null_indices, cat_col]
df.loc[null_indices, cat_col] = None

# Create a mask for the non-null values of the categorical column
not_null_mask = df[cat_col].notnull()

# Train an SVM classifier on the non-null values of the categorical column
X_train = df[num_cols][not_null_mask] # CHANGE to rather: num + already_done_categorical cols
y_train = df[cat_col][not_null_mask]

model = KNeighborsClassifier()
model.fit(X_train, y_train)

# Predict the categorical column values for the rows with null values
X_test = df[num_cols].iloc[null_indices]
y_pred = model.predict(X_test)

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"test length: {len(y_pred)}")
# Print the results
print(f"Results for {cat_col}:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")

In [ ]:
y_pred

In [ ]:
df[num_cols].iloc[null_indices]

In [ ]:
preprocessor.fit_transform(X_train)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
display_side_by_side(X_train, pd.DataFrame(y_train))

!/usr/share/miniconda2/envs/py39/bin/pip install fancyimpute --user

In [ ]:
df=df_dict[mice_category_id]

In [ ]:
df_dict[mice_category_id].iloc[:10,:]

#  category MICE

##  columns

In [ ]:
df_dict[mice_category_id].columns

##  taking a look

In [ ]:
df=df_dict[mice_category_id]
df.head(50)

In [ ]:
df[~df["Pointing device.440"].isnull()]

# get each of its feature's name, unit, possible values (if restricted)

In [ ]:
# Parse the XML file
feature_tree = ET.parse(feature_xml_filepath)
config.feature_root = feature_tree.getroot()
column_metadata=get_column_metadata_for_category(mice_category_id)
print_first_x_dict_elements(column_metadata, 5)     

# validate, can we find all colums in the dataset, in the metadata listed features of the category?

## create dictionary of features in the specific category's feather file

In [ ]:
dataset_existing_columns = {}
for column in df.columns:
    parts = column.split(".")
    if len(parts) > 1 and not column.endswith(".unit"):
        key = parts[-1]
        if key.isnumeric():
            value = ".".join(parts[:-1])
            dataset_existing_columns[int(key)] = value
print(f"{len(dataset_existing_columns)} columns are in the dataset without a .unit in it.\n")
print_first_x_dict_elements(dataset_existing_columns, 5)     

In [ ]:
dataset_cols = set(dataset_existing_columns.keys())
category_cols = set(column_metadata.keys())


missing_keys = dataset_cols.difference(category_cols)

if missing_keys:
    print("The following features from the dataset are not present in the metadata for this category:")
    for key in missing_keys:
        print(key, dataset_existing_columns[key])
else:
    print("All features from the dataset are present in the metadata for this category.")

In [ ]:
"Recycled material" in df.columns

In [ ]:
for i in df.columns: print(i)

# data dictionary

In [ ]:
tree = ET.parse(data_dict_filepath)
root = tree.getroot()

In [ ]:
for child in root:
    print("\n-------------------------------------------------------------\nCategory:", child.tag)
    count = 0
    for subchild in child:
        print("\t", subchild.tag, subchild.attrib,)
        count += 1
        if count == 5:
            break

In [ ]:
for child in root:
    print("Category:", child.tag)
    count = 0
    for subchild in child:
        print("\tSubcategory:", subchild.tag, subchild.attrib)
        subcount = 0
        for subsubchild in subchild:
            print("\t\tElement:", subsubchild.tag, subsubchild.attrib)
            print("\t\tContent:", subsubchild.text)
            subcount += 1
            if subcount == 3:
                break
        count += 1
        if count == 3:
            break

# graphs about missing value combinations for each category, similary as I've on MICE
# de ahol pl. nincs 30db vagy nemtom hány minta, azt alapból ignorálni... too small sample size?

1000 rows of data in a category
remove....

They are the real data they have in their systems. 
If there are missing data for something it might actually be missing (or it could be on purpose), but as Shabana, Wael E says, you can remove some data randomly and try imputation techniques for these.
